In [6]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

from api_clients import SparqlClient

pd.options.display.float_format = '{:,.2f}'.format

In [90]:
client = SparqlClient()
query = """
    PREFIX or: <http://openresearch.org/vocab/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>

    SELECT DISTINCT ?politician ?party ?job ?org_label ?role ?employmentType ?remuneration
    WHERE {

        ?mep_uri a dbo:MemberOfParliament;
               schema:memberOf ?party_uri;
               schema:familyName ?lastName;
               schema:givenName ?firstName.
        OPTIONAL {
        ?mep_uri schema:hasOccupation ?job_uri.
        ?job_uri <http://schema.org/name> ?job.	
        FILTER(LANG(?job) = "de")
      }

        ?party_uri a dbo:PoliticalParty;  
          schema:name ?party.

        OPTIONAL{
          ?compensation a lw:MembersInterestsCompensation;
              <http://www.w3.org/ns/org#memberOf> ?interests;
              <http://www.w3.org/ns/org#remuneration> ?remuneration.

          ?interests a <https://lod.lobbywatch.ch/MembersInterest>;
                   <https://lod.lobbywatch.ch/parliamentMember> ?mep_uri;
                  <http://www.w3.org/ns/org#organization> ?org;
                  <http://www.w3.org/ns/org#role> ?role;
                  <https://lod.lobbywatch.ch/employmentType> ?employmentType.

          ?org <http://schema.org/name> ?org_label.
          FILTER(LANG(?org_label) = "de")
        }

        BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .        
        FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician ?org_label
"""

df = client.send_query(query)
df = df.groupby(["politician", "party", "job", "org_label", "employmentType", "remuneration"], dropna=False).agg(lambda x: set(x)).reset_index()
df.head()

,politician,party,job,org_label,employmentType,remuneration,role
0,Ada Marra,Sozialdemokratische Partei,Nationalrätin,Fondation Marie-Eléonore d'Olcah,part-time,0.00,"{beirat, mitglied}"
1,Ada Marra,Sozialdemokratische Partei,Nationalrätin,Fondation Mère Sofia,part-time,0.00,"{vorstand, praesident}"
2,Ada Marra,Sozialdemokratische Partei,Nationalrätin,Nationale Plattform zu den Sans-Papiers,part-time,0.00,"{vorstand, praesident}"
3,Ada Marra,Sozialdemokratische Partei,Nationalrätin,Sozialdemokratische Partei der Schweiz,part-time,"5,000.00","{vizepraesident, geschaeftsfuehrend}"
4,Adèle Thorens Goumaz,Grüne Partei der Schweiz,"Beraterin, Erwachsenenbildnerin",Alliance suisse pour une agriculture sans géni...,part-time,0.00,"{vizepraesident, vorstand}"


In [93]:
# List of politicians that did not disclose their income
disclosures = df[["politician", "party", "remuneration"]].groupby(["politician", "party"]).agg(lambda x: len(set(x).difference(set([1, float("nan")]))) != 0).reset_index().rename(columns={"remuneration": "has_disclosed_income"})
disclosures[~disclosures["has_disclosed_income"]].sort_values(by=["party"])

,politician,party,has_disclosed_income
207,Ruth Humbel,Christlichdemokratische Volkspartei,False
185,Philipp Kutter,Christlichdemokratische Volkspartei,False
175,Nicolo Paganini,Christlichdemokratische Volkspartei,False
128,Lilian Studer,Evangelische Volkspartei,False
3,Alex Farinelli,Freisinnig-Demokratische Partei,False
187,Philippe Bauer,Freisinnig-Demokratische Partei,False
106,Jacques Bourgeois,Freisinnig-Demokratische Partei,False
152,Martin Schmid,Freisinnig-Demokratische Partei,False
17,Anna Giacometti,Freisinnig-Demokratische Partei,False
105,Jacqueline de Quattro,Freisinnig-Demokratische Partei,False


In [94]:
shame_table = disclosures.drop(columns = "politician").groupby(["party"]).agg([np.mean, len, lambda x: len(x) - sum(x)]).reset_index()
shame_table.columns = ["party", "ratio_disclosed", "members", "nondisclosing_members"]
shame_table = shame_table.sort_values(by=["ratio_disclosed", "members"])

shame_table

,party,ratio_disclosed,members,nondisclosing_members
5,Evangelische Volkspartei,0.67,3,1
12,Schweizerische Volkspartei,0.80,59,12
6,Freisinnig-Demokratische Partei,0.85,40,6
3,Christlichdemokratische Volkspartei,0.92,37,3
8,Grünliberale Partei,0.94,16,1
7,Grüne Partei der Schweiz,0.97,31,1
0,Basels starke Alternative,1.00,1,0
2,CVP Oberwallis,1.00,1,0
4,Eidgenössisch-Demokratische Union,1.00,1,0
9,Lega dei Ticinesi,1.00,1,0


In [95]:

fig = px.bar(shame_table, x="ratio_disclosed", y="party", hover_data=["members", "nondisclosing_members"], orientation='h')
fig.show()

In [115]:
query = """
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?politician ?party ?org ?org_uri ?org_id
    WHERE {

      ?mep a dbo:MemberOfParliament;
          schema:memberOf ?party_uri;
          schema:familyName ?lastName;
          schema:givenName ?firstName.

      BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .

      ?party_uri a dbo:PoliticalParty;
                 skos:altLabel ?party.

      ?interests a lw:MembersInterest;
        org:organization ?org_uri;
        lw:parliamentMember ?mep.

      OPTIONAL{
        ?org_uri <http://www.w3.org/ns/org#identifier> ?org_id
      }
      ?org_uri schema:name ?org.

      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician
"""

direct_influence = client.send_query(query)

In [121]:
query = """
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>

    SELECT ?politician ?party ?org ?org_uri ?org_id
    WHERE {

      ?mep a dbo:MemberOfParliament;
          schema:memberOf ?party_uri;
          schema:familyName ?lastName;
          schema:givenName ?firstName.

      BIND(CONCAT(?firstName, " ", ?lastName) AS ?politician) .

      ?party_uri a dbo:PoliticalParty;
                 skos:altLabel ?party.

      ?access a lw:AccessRight;
          lw:issuedBy ?mep;
          lw:issuedTo ?person.
      ?person org:hasMembership ?membership .
      ?membership org:organization ?org_uri.
      ?org_uri schema:name ?org.

      OPTIONAL{
        ?org_uri <http://www.w3.org/ns/org#identifier> ?org_id.
      }

      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
    ORDER BY ?politician
"""

indirect_influence = client.send_query(query)

In [123]:
all_influences = pd.concat([direct_influence, indirect_influence])

In [151]:
df = direct_influence #all_influences, direct_influence, indirect_influence

orgs_influence = df[["org", "org_uri", "politician"]].groupby(["org", "org_uri"], dropna=False).count().reset_index().sort_values(by=["politician"], ascending=False)
orgs_influence.columns = ["org", "org_uri", "influenced_politicians"]
orgs_influence[0:20]

,org,org_uri,influenced_politicians
1892,Sport,https://lod.lobbywatch.ch/organisation/1751,140
2359,Wohn- und Grundeigentum,https://lod.lobbywatch.ch/organisation/2899,92
223,Biodiversität und Artenschutz,https://lod.lobbywatch.ch/organisation/637,89
217,"Bildung, Forschung und Innovation",https://lod.lobbywatch.ch/organisation/2124,88
252,Bundeshaus-Band,https://lod.lobbywatch.ch/organisation/1750,81
152,Auslandschweizer,https://lod.lobbywatch.ch/organisation/1278,79
810,"Gruppa parlamentara ""lingua e cultura rumantscha""",https://lod.lobbywatch.ch/organisation/5964,78
2356,Wirtschafts- und währungspolitischer Arbeitskreis,https://lod.lobbywatch.ch/organisation/5978,65
1554,Rotes Kreuz,https://lod.lobbywatch.ch/organisation/5024,61
479,Erneuerbare Energien,https://lod.lobbywatch.ch/organisation/1505,58


In [152]:
# Number of members in each party:
parties = direct_influence[["politician", "party"]].drop_duplicates().groupby("party").count().reset_index().sort_values(by="politician", ascending = False)
parties[parties["politician"] > 5]

,party,politician
13,SVP,59
12,SP,48
6,FDP,40
2,CVP,37
8,GPS,31
7,GLP,16


In [153]:
relevant_parties = list(parties[parties["politician"] > 5]["party"])

In [154]:
indirect_influence[["politician","party", "org"]]

,politician,party,org
0,Adèle Thorens Goumaz,GPS,Greenpeace Schweiz
1,Adèle Thorens Goumaz,GPS,Agrarallianz
2,Adèle Thorens Goumaz,GPS,WWF Schweiz
3,Albert Rösti,SVP,Berner Bauern Verband Management AG
4,Albert Rösti,SVP,Kanton Bern
...,...,...,...
1548,Yvonne Feri,SP,IV-Allianz
1549,Yvonne Feri,SP,Schweizerischer Musikpädagogischer Verband
1550,Yvonne Feri,SP,Allianz Ergänzungsleistungen
1551,Yvonne Feri,SP,Inclusion Handicap


In [155]:
df = indirect_influence #all_influences, direct_influence, indirect_influence

influences = pd.pivot_table(df[["politician","party", "org"]], values='politician', index=['org'], columns=['party'], aggfunc=len).fillna(0)
influences["total"] = influences.sum(axis=1).astype(int)

influences = influences[["total"] + relevant_parties]
influences[relevant_parties] = influences[relevant_parties].div(influences.total, axis = 0)
influences.sort_values(by="total", ascending = False)[10:30]

party,total,SVP,SP,FDP,CVP,GPS,GLP
org,,,,,,,
economiesuisse,5,0.20,0.00,0.40,0.20,0.00,0.20
furrerhugi.ag,5,0.00,0.20,0.40,0.20,0.00,0.00
Bündnis Freiheitliches Gesundheitswesen,5,0.80,0.00,0.00,0.00,0.00,0.00
Sozialdemokratische Partei der Schweiz,4,0.00,0.25,0.00,0.00,0.75,0.00
Reformierte Kirchen Bern-Jura-Solothurn,4,0.00,0.50,0.25,0.00,0.00,0.00
Amnesty International,4,0.00,1.00,0.00,0.00,0.00,0.00
Inclusion Handicap,4,0.00,0.50,0.00,0.00,0.25,0.00
Greenpeace Schweiz,4,0.00,0.25,0.00,0.00,0.75,0.00
Agrarallianz,4,0.00,0.25,0.00,0.00,0.75,0.00


In [156]:
query = """
    PREFIX http: <http://www.w3.org/2011/http#>
    PREFIX or: <http://openresearch.org/vocab/>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX schema: <http://schema.org/>
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX lw: <https://lod.lobbywatch.ch/>
    PREFIX org: <http://www.w3.org/ns/org#>

    SELECT DISTINCT ?lobbyName ?sectorName ?org ?lobbyist ?party
    WHERE {
      ?lobby a lw:Lobby;
         rdfs:label ?lobbyName;
        lw:businessSector ?sector ;
        schema:member ?member .
      ?sector rdfs:label ?sectorName .

      ?member schema:familyName ?lastName;
              schema:givenName ?firstName.

      OPTIONAL{
        ?access lw:issuedTo ?member.
        ?access lw:issuedBy ?politician.
        ?politician schema:memberOf ?political_org.
        ?political_org a dbo:PoliticalParty.
        ?political_org <http://www.w3.org/2004/02/skos/core#altLabel> ?party.

        ?member schema:hasOccupation ?occupation_uri;
               org:hasMembership ?membership.
        ?membership <http://www.w3.org/ns/org#organization> ?org_uri.
        ?org_uri <http://schema.org/name> ?org.
      }
      BIND(CONCAT(?firstName, " ", ?lastName) AS ?lobbyist) .

      FILTER(LANG(?lobbyName) = "de")
      FILTER(LANG(?sectorName) = "de")
      FILTER(LANG(?org) = "de")
      FILTER(LANG(?party) = "de")
    }
"""

df = client.send_query(query)

In [166]:
lobbies = dict()
for result in results:

    org = result["org"].value
    subsector = result["lobbyName"].value
    sector = result["sectorName"].value

    if sector in lobbies:
        
        if subsector in lobbies[sector]:
            lobbies[sector][subsector].add(org)
        else:
            lobbies[sector][subsector] = set([org])
        
    else:
        lobbies[sector] = {
            subsector: set([org]),
        }
    

In [167]:
# Companies having its stakes in lobby
lobbies["Gesundheit"]["Krankenkassen"]

{'ARA Kiesental-Aaretal ARAKA AG',
 'Agroscope',
 'BWZ Berufs- und Weiterbildungszentrum Lyss',
 'CSS Versicherung AG',
 'EAE (Energieagentur Elektrogeräte)',
 'Gemeinde Konolfingen',
 'Groupe Mutuel Services SA',
 'Pink Cross',
 'Santésuisse',
 'Schweizerische Volkspartei',
 'Schweizerische Volkspartei International',
 'Swiss Diversity Award',
 'digital.swiss'}

In [168]:
subset = df[["lobbyName", "lobbyist", "sectorName"]].drop_duplicates().groupby(["lobbyName", "sectorName"]).count().reset_index().sort_values(by="lobbyist", ascending=False)
subset[0:10]

,lobbyName,sectorName,lobbyist
16,Consulting/PR,Wirtschaft,40
4,Arbeitnehmerorganisationen,Wirtschaft,14
68,Umwelt & Natur,Umwelt,12
13,Bildung/Wissenschaft,Bildung,11
39,Kantone/Regionen,Staatspolitik/Staatswirtschaft,10
67,Tourismus/Gastronomie,Wirtschaft,8
2,Advokaturen/Treuhand,Wirtschaft,8
44,Landwirtschaft allgemein,Landwirtschaft,8
38,KMU/Gewerbe/Arbeitgeber,Wirtschaft,8
8,Banken,Wirtschaft,7


In [169]:
lobbies["Wirtschaft"]["Banken"]

{'Bank Julius Bär & Co. AG',
 'Bündner Offiziersgesellschaft',
 'CVP Baden',
 'FDP Die Liberalen Fällanden',
 'FDP Die Liberalen Graubünden',
 'FDP Die Liberalen Schweiz',
 'FDP die Liberalen Baar',
 'Freidenker-Vereinigung der Schweiz',
 'Gemeinde Fällanden',
 'HarbourClub, Chief Communications Officers',
 'Jungfreisinnige Graubünden',
 'LZ Medien Holding AG',
 'Raiffeisen Schweiz',
 'SAMDalumni',
 'SIX Group AG',
 'SOG Schweizerische Offiziersgesellschaft',
 'Schweizerische Bankiervereinigung',
 'Schweizerische Offiziersgesellschaft der Artillerie',
 'Stadt Baden',
 'Surseer Woche AG',
 'Universität Luzern',
 'Verband Schweizerischer Kantonalbanken',
 'Vereinigung Schweizerischer Assetmanagement- und Vermögensverwaltungsbanken VAV',
 'Vereinigung Schweizerischer Privatbanken'}

In [170]:
parties = df[["lobbyName", "sectorName", "lobbyist", "party"]].drop_duplicates()
lobbysts = pd.pivot_table(parties, values='lobbyist', index=['lobbyName', "sectorName"], columns=['party'], aggfunc=len).fillna(0)
lobbysts["total"] = lobbysts.sum(axis=1)

lobbysts = lobbysts[["total"] + relevant_parties].astype(int)
lobbysts.sort_values(by="total", ascending = False)[0:10]

,party,total,SVP,SP,FDP,CVP,GPS,GLP
lobbyName,sectorName,,,,,,,
Consulting/PR,Wirtschaft,40,10,11,8,5,1,3
Arbeitnehmerorganisationen,Wirtschaft,14,0,10,0,0,4,0
Umwelt & Natur,Umwelt,12,0,2,0,0,9,0
Bildung/Wissenschaft,Bildung,11,1,6,2,2,0,0
Kantone/Regionen,Staatspolitik/Staatswirtschaft,10,1,2,2,2,2,0
Tourismus/Gastronomie,Wirtschaft,8,2,0,0,5,1,0
Advokaturen/Treuhand,Wirtschaft,8,3,0,3,2,0,0
Landwirtschaft allgemein,Landwirtschaft,8,4,0,1,3,0,0
KMU/Gewerbe/Arbeitgeber,Wirtschaft,8,2,0,4,2,0,0


In [171]:
party = "SP"
all_influences[all_influences["party"] == party]["org"].drop_duplicates()[0:10]

0                                 Frauengruppe
1      Nationale Plattform zu den Sans-Papiers
2                        Pfadibewegung Schweiz
3                              Bundeshaus-Band
4                         Fondation Mère Sofia
5       Sozialdemokratische Partei der Schweiz
6             Fondation Marie-Eléonore d'Olcah
349                                  Migration
351                      Amnesty International
352         Homosexuelle Arbeitsgruppen Zürich
Name: org, dtype: object

In [172]:
company = "Amnesty International"

all_influences[(all_influences["party"] == party) & (all_influences["org"] == company)]

,politician,party,org,org_uri,org_id
351,Angelo Barrile,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
505,Barbara Gysi,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
1289,Cédric Wermuth,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
3831,Martina Munz,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
4249,Nadine Masshardt,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
4898,Roger Nordmann,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
5559,Ursula Schneider Schüttel,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
259,Christian Dandrès,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
837,Laurence Fehlmann Rielle,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
1033,Mathias Reynard,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410


In [173]:
indirect_influence[(indirect_influence["party"] == party) & (indirect_influence["org"] == company)]

,politician,party,org,org_uri,org_id
259,Christian Dandrès,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
837,Laurence Fehlmann Rielle,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
1033,Mathias Reynard,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
1547,Yvonne Feri,SP,Amnesty International,https://lod.lobbywatch.ch/organisation/1064,CHE-106.126.410
